# Foundations of Computer Science - progetto 2022/23

- Mattia Birti [897092] 
- Alberto Porrini [826306]
- Gloria Longo [864579]

You have to work on the Dogs adoptions dataset:
*  [Dogs](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogs.csv)
*  [Dog Travel](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogTravel.csv)
*  [NST-EST2021-POP](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/NST-EST2021-POP.csv)

### Notes

1. It is mandatory to use GitHub for developing the project.
1. The project must be a jupyter notebook.
1. There is no restriction on the libraries that can be used, nor on the Python version.
1. All questions on the project must be asked in a public channel on Zulip.
1. At most 3 students can be in each group. You must create the groups by yourself.
1. You do not have to send me the project before the discussion.s

## Libraries

In [1]:
import pandas as pd
from datetime import datetime
from difflib import SequenceMatcher

### Import data from GitHub

#### Dogs.csv

In [2]:
dogs = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogs.csv", sep=",", encoding='latin-1')

In [ ]:
dogs.shape

In [ ]:
dogs.head()

#### DogTravel.csv

In [ ]:
dogTravel = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogTravel.csv", sep=",", encoding='latin-1')

In [ ]:
dogTravel.shape

In [ ]:
dogTravel.head()

#### NST-EST.csv

In [ ]:
nstest = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/NST-EST2021-POP.csv", header=None, sep=",", encoding='latin-1')

In [ ]:
nstest.shape

In [ ]:
nstest.head()

## 1. Extract all dogs with status that is not adoptable

Explore the dog's column names

In [ ]:
dogs.columns

Explore the first row

In [ ]:
dogs.iloc[0,:]

We can see that there are two variables (posted, accessed) that contain dates, let's check if they are in the right format

In [ ]:
type(dogs.loc[0, 'posted'])
type(dogs.loc[0, 'accessed'])

Since they are in string format, we convert them to date format

In [ ]:
#for i in range(len(dogs)):
        #dogs.loc[i, 'posted']=datetime.strptime(dogs.loc[i,'posted'], "%Y-%m-%dT%H:%M:%S+0000")
        #dogs.loc[i,'accessed']=datetime.strptime(dogs.loc[i,'accessed'], "%Y-%m-%d")

We can see that there are some variables that do not contain a date, let's see which ones

In [ ]:
for i in range(len(dogs)):
    if(len(dogs.loc[i,'posted'])!=24):
        print(dogs.iloc[i,])

The problem with these rows is that they have not been split properly in cell 'name'. Let's solve the problem and transform the dates into datatime format

In [ ]:
for i in range(len(dogs)):
    if(len(dogs.loc[i,'posted'])==24):
        dogs.loc[i, 'posted']=datetime.strptime(dogs.loc[i,'posted'], "%Y-%m-%dT%H:%M:%S+0000")
        dogs.loc[i,'accessed']=datetime.strptime(dogs.loc[i,'accessed'], "%Y-%m-%d")
    else :
        j=26
        l=(dogs.iloc[i,24]).split(' ',1)
        prov=dogs.iloc[i,25]
        dogs.iloc[i,25]=l[1]
        dogs.iloc[i,24]=l[0]
        while(j<34):
            prov2=dogs.iloc[i,j]
            dogs.iloc[i,j]=prov
            prov=prov2
            j=j+1
        dogs.loc[i, 'posted']=datetime.strptime(dogs.loc[i,'posted'], "%Y-%m-%dT%H:%M:%S+0000")
        dogs.loc[i,'accessed']=datetime.strptime(dogs.loc[i,'accessed'], "%Y-%m-%d")

Now we can solve this task using a 'while' loop which scrolls through all the rows of the dataframe and stores only the identification code of the dogs that have a status other than adoptable

In [ ]:
lista=[]
i=0
while(i<len(dogs)):
    if (dogs.loc[i,'status']!='adoptable'):
        lista.append(dogs.loc[i,'id'])
    i=i+1
lista

## 2. For each (primary) breed, determine the number of dogs

In [ ]:
dogs.groupby('breed_primary').count()['id']

## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

Let's see all the primary breed with a groupBy:

In [ ]:
nBreed = dogs.groupby('breed_primary').count()[['species','breed_secondary']]
nBreed

As we can see we can use "species" as total number of individues for this species.
Affenpischer are in total 17 dogs, 2 of those are mixed breed.

Rename colums

In [ ]:
nBreed = nBreed.rename( columns = {'species':'total', 'breed_secondary':'nSecondary'})
nBreed

We have to compute how many are primary breed:

In [ ]:
nBreed['nPrimary'] = nBreed['total'] - nBreed['nSecondary']
nBreed

It's time to compute the ratio between Primary and Secondary Breed.

In [ ]:
nBreed['ratioBreed'] = nBreed['nPrimary'] / nBreed['nSecondary']
nBreed

As we can see, there are many 'inf' values. It is correct because for that breed there are not any secondary breed.

## 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

We use the group by method to solve this task starting from the latest posted timestamp.

In [ ]:
df_new = dogs[dogs.groupby('breed_primary')['posted'].transform('max') == dogs['posted']]
df_new.loc[ :, ['breed_primary', 'posted']]

In [ ]:
df_new = dogs[dogs.groupby('breed_primary')['posted'].transform('min') == dogs['posted']]
df_new.loc[ :, ['breed_primary', 'posted']]

## 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

we create a table with count of male dogs for each state (contanct_state) 

In [ ]:
dog_male=dogs[dogs['sex']=='Male']
dog_male=dog_male.groupby(['contact_state'])[['sex']].count()
dog_male.tail()

we recreate the same table that we have done before with female dogs for each state

In [ ]:
dog_female=dogs[dogs['sex']=='Female']
dog_female=dog_female.groupby(['contact_state'])[['sex']].count()
dog_female.tail()

we create a new table call "dog_sex" thanks to merge between male dogs and female dogs on contact_state

In [ ]:
dog_sex=pd.merge(dog_male,dog_female,on="contact_state")
dog_sex.rename( columns = {'sex_x':'tot_male','sex_y':'tot_female'})
dog_sex=dog_sex.rename( columns = {'sex_x':'tot_male','sex_y':'tot_female'})
dog_sex.tail()

we compute the imbalance like difference between male and female dogs and after we calculate the absolute value of the imbalance given that we consider only positive value of imbalance for each contact_state

In [ ]:
dog_sex["imbalance"]=dog_sex["tot_male"]-dog_sex["tot_female"]
dog_sex["imbalance"]=abs(dog_sex["imbalance"])
dog_sex.tail()

we found the state with the largest imbalance

In [ ]:
dog_sex[dog_sex["imbalance"]==dog_sex["imbalance"].max()]

In [ ]:
dog_sex["imbalance"].nlargest(1)

## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

Explore the dog's column names

In [ ]:
dogs.columns

Compute the mean of the stay duration and stay cost for the pair AGE, SIZE;

In [ ]:
AgeSize = dogs.groupby(['age', 'size'])[['stay_duration','stay_cost']].mean()
AgeSize

## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

Let's start grouping the dataset by the variable id

In [ ]:
count=dogTravel.groupby('id').count()

We reset the index

In [ ]:
count.reset_index(inplace=True)

Now we compute a list of only dogs that have made at least 2 trips

In [ ]:
list_index=(count[count['index']>2])['id'].tolist()

Finally we build a dictionary having as key the identification code of the dog that has made at least 2 trips, and as value its primary breed. 
To do this a for loop will be used which compares each identification code of the dog in the staring dataset with those contained in the list created above



In [ ]:
diz={}
for index in list_index:
    for i in range(len(dogs)):
        if (dogs.loc[i, 'id']==index):
            diz[dogs.loc[i, 'id']]= dogs.loc[i, 'breed_primary']
diz

## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

To resolve the question, we create two different tables, the first table is compose with only observation field manual full.

In [ ]:
manual_not_na=dogTravel[dogTravel['manual'].notna()]
manual_not_na

in this table we create a new column ("correct_state") with the same values in manual (cause are the values that will be overrides in found column)

In [ ]:
manual_not_na["correct_state"] = manual_not_na["manual"]
manual_not_na

on second table, we subset only the observations with empty manual values

In [ ]:
manual_na=dogTravel[dogTravel['manual'].isna()]
manual_na

in this table we create a new column ("correct_state") with the same values in found (cause are the values that will be not overrides)

In [ ]:
manual_na["correct_state"] = manual_na["found"]
manual_na

subsequantly we recreate a new df with the concatenate of the two tables that we had created before, and we order them with index sort

In [ ]:
df=pd.concat([manual_na,manual_not_na])
df=df.sort_index()
df

now we have got a column correct_state complete with the values overrides and not overrides, so we replace that values at the column found of dogTravel dataframe and we ending the point

In [ ]:
dogTravel["found"]=df["correct_state"]
dogTravel

In the last result, we have got in found column the manual values only if they are not "NaN" 

## 9. For each state, compute the ratio between the number of travels and the population.

In [ ]:
import pycountry #have to install it (pip install pycountry)

### First Dataset Analysis

In [ ]:
dogTravel.shape #number of total travell done

In [ ]:
dogTravel.groupby(['contact_state']).count().shape 
#number of countries that have made trips

In [ ]:
nstest.shape #number of country in csv population

51 states, 45 states that have made trips, total trips 6149

In [ ]:
#change label
nTravelForState = dogTravel.groupby(['contact_state']).count()[['index']].reset_index().rename(columns={'contact_state':'state', 'index':'nTravel'})
nTravelForState.head()

In [ ]:
nTravelForState['nTravel'].sum() #sum is correct

In [ ]:
nTravelForState.shape

### There is an error

In [ ]:
dogTravel[dogTravel['contact_state'] == '17325']

The number refers to state of Pensylvania. It needs to be corrected

In [ ]:
nTravelForState[nTravelForState['state']=='PA']

In [ ]:
#take the 2 values
PAerr = nTravelForState[nTravelForState['state'] == '17325']['nTravel']
PAcorr = nTravelForState[nTravelForState['state'] == 'PA']['nTravel']

In [ ]:
#delete the wrong row
nTravelForState = nTravelForState.drop(nTravelForState.index[nTravelForState['state'] == '17325']).reset_index()

In [ ]:
nTravelForState.head()

In [ ]:
#Add the right value
nTravelForState.loc[nTravelForState['state']=='PA', 'nTravel'] = sum(PAcorr, PAerr).values
nTravelForState[nTravelForState['state'] == 'PA']

#### Country that have made trips become 43

### Table State and Popolation

In [ ]:
nstest[1] = nstest[1].str.replace('.','')

In [ ]:
state = nstest.rename(columns = {0:'state', 1:'nPopulation'})
state.head()

#### Add state code

In [ ]:
def findCountryAlpha2 (country_name):
    try:
        sub = pycountry.subdivisions.lookup(country_name)
        sample_str = sub.code
        stateCode = sub.code[-2:]
        return stateCode
    except:
        return ("not founded!")

state['state_code'] = state.apply(lambda row: findCountryAlpha2(row.state) , axis = 1)
state.head()

### Still errors

#### MT

In [ ]:
#Library pycountry has a code wrong for Montana state
state[state['state']=='Montana']

In [ ]:
state.loc[state['state']=='Montana', 'state_code'] = 'MT'
state[state['state']=='Montana']

#### MY

In [ ]:
# another error in library pycountry referred to Maryland code state
state[state['state']=='Maryland']

In [ ]:
state.loc[state['state']=='Maryland', 'state_code'] = 'MD'
state[state['state']=='Maryland']

#### CSV nstest

In [ ]:
#there are some state_code NB which are the same state that NJ
nTravelForState[nTravelForState['state']=='NB']

In [ ]:
nTravelForState[nTravelForState['state']=='NJ']

In [ ]:
NJerr = nTravelForState[nTravelForState['state'] == 'NB']['nTravel']
NJcorr = nTravelForState[nTravelForState['state'] == 'NJ']['nTravel']

In [ ]:
#delete the wrong row
nTravelForState = nTravelForState.drop(nTravelForState.index[nTravelForState['state'] == 'NB']).reset_index()

In [ ]:
#salvo il valore in NJ
nTravelForState.loc[nTravelForState['state']=='NJ', 'nTravel'] = sum(NJcorr, NJerr).values
nTravelForState[nTravelForState['state'] == 'NJ']

#### Now the number of country that have done trips are 43

### Merging time

In [ ]:
table_travel_for_state = pd.merge(state, nTravelForState, left_on='state_code', right_on='state')[['state_x', 'nPopulation', 'nTravel']]
table_travel_for_state.head()

In [ ]:
#check if sum is correct
table_travel_for_state['nTravel'].sum()

### Add ratio between the number of travels and the population

In [ ]:
table_travel_for_state['nPopulation'] = table_travel_for_state['nPopulation'].apply(pd.to_numeric)

In [ ]:
table_travel_for_state['ratio_Travel_Population'] = table_travel_for_state['nPopulation'] / table_travel_for_state['nTravel']
table_travel_for_state.head()

## 10. For each dog, compute the number of days from the posted day to the day of last access.

To solve this task the difference between the column 'posted' and the column 'accessed' will be calculated.
The information obtained will be stored in a dictionary having the dog's identification code as the key and 
the days between the posted day and the last access as value.



In [ ]:
diz={}
for i in range(len(dogs)):
    diz[dogs.loc[i,'id']]= -1*(dogs.loc[i, 'posted'] - dogs.loc[i, 'accessed']).days
diz

## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [3]:
# 1) creo una tabella con tutti gli elementi che hanno uguale breeed & sex 
duplicate_table = dogs.iloc[:, [13,5,34]]
duplicate_table

,sex,breed_primary,description
0,Male,American Staffordshire Terrier,Harley is not sure how he wound up at shelter ...
1,Male,Pit Bull Terrier,6 year old Biggie has lost his home and really...
2,Male,Shepherd,Approx 2 years old.\n Did I catch your eye? I ...
3,Female,German Shepherd Dog,NaN
4,Male,Dachshund,Theo is a friendly dachshund mix who gets alon...
...,...,...,...
58175,Male,Border Collie,"Due to the small size of our volunteer base, w..."
58176,Female,Australian Shepherd,NaN
58177,Female,Border Collie,"Due to the small size of our volunteer base, w..."
58178,Male,Boxer,NaN


In [5]:
#elimino tutte le righe che hanno NaN come descrizione perché non possono essere duplicate 
duplicate_table = duplicate_table[duplicate_table['description'].notna()]

,sex,breed_primary,description
6,Female,Italian Greyhound,Macadamia was born around July 8th and donÃ¢Â...
8,Female,Cattle Dog,Huckleberry is a friendly girl who was born Ma...
10,Female,Cattle Dog,Speckles is a 4 month old Heeler/Hound mix who...
11,Female,Italian Greyhound,Cashew is sweet girl who was born around July ...
13,Female,Border Collie,Meet Sydney- SheÃ¢ÂÂs a 5 year old Border Co...
...,...,...,...
58145,Female,American Staffordshire Terrier,Bella is a very sweet dog. She would do best i...
58147,Female,Labrador Retriever,Please stop by any day between Noon and 2pm or...
58149,Female,Labrador Retriever,Please stop by any day between Noon and 2pm or...
58160,Female,American Staffordshire Terrier,Our Baby Girl has had a rough life: bred every...


In [4]:
duplicate_female = duplicate_table[duplicate_table['sex'] == 'Female']
duplicate_male = duplicate_table[duplicate_table['sex'] == 'Male']
#duplicate_female

,sex,breed_primary,description
3,Female,German Shepherd Dog,NaN
6,Female,Italian Greyhound,Macadamia was born around July 8th and donÃ¢Â...
8,Female,Cattle Dog,Huckleberry is a friendly girl who was born Ma...
10,Female,Cattle Dog,Speckles is a 4 month old Heeler/Hound mix who...
11,Female,Italian Greyhound,Cashew is sweet girl who was born around July ...
...,...,...,...
58168,Female,Black Labrador Retriever,NaN
58173,Female,Labrador Retriever,NaN
58174,Female,Shepherd,NaN
58176,Female,Australian Shepherd,NaN


In [6]:
#riordino le righe per breed-primary (una sort di groupby)
duplicate_female = duplicate_female.sort_values(by = 'breed_primary').reset_index()
duplicate_male = duplicate_male.sort_values(by = 'breed_primary').reset_index()
#duplicate_female

,index,sex,breed_primary,description
0,32156,Female,Affenpinscher,Elsie.jpgThank you for looking at our availabl...
1,54676,Female,Affenpinscher,Sweet older girl whose had a rough couple year...
2,24994,Female,Affenpinscher,To see even more click the following link:\n\n...
3,51866,Female,Affenpinscher,Norma Jean is an Affenpinscher mix girl that w...
4,32235,Female,Affenpinscher,55351.jpgThank you for looking at our availabl...
...,...,...,...,...
23825,21554,Female,Yorkshire Terrier,You can fill out an adoption application onlin...
23826,37159,Female,Yorkshire Terrier,You can fill out an adoption application onlin...
23827,12537,Female,Yorkshire Terrier,"PRECIOUS is a 9 year old, 7 pound purebred Yor..."
23828,6734,Female,Yorkshire Terrier,Gidget is only 9 months and about 4 lbs. She's...


In [7]:
duplicate_female2 = duplicate_female.iloc[0:200]
duplicate_female2

,index,sex,breed_primary,description
0,32156,Female,Affenpinscher,Elsie.jpgThank you for looking at our availabl...
1,54676,Female,Affenpinscher,Sweet older girl whose had a rough couple year...
2,24994,Female,Affenpinscher,To see even more click the following link:\n\n...
3,51866,Female,Affenpinscher,Norma Jean is an Affenpinscher mix girl that w...
4,32235,Female,Affenpinscher,55351.jpgThank you for looking at our availabl...
...,...,...,...,...
195,42145,Female,American Bulldog,Surely youÃ¢ÂÂve seen DottieÃ¢ÂÂs beautifu...
196,24570,Female,American Bulldog,Hi! My name is Cash. I am a very loving and af...
197,35850,Female,American Bulldog,*This dog IS available (unless photo shows ado...
198,28017,Female,American Bulldog,Dumpling was found as a stray shortly after sh...


In [8]:
def scannerDuplicate(tab): # TROPPO LENTO ESPONENZIALE
    duplicate = {}
    for i in range(len(tab)): # per scorrere tutte le righe
        breed1 = tab.iloc[i]['breed_primary']
        desc1 = tab.iloc[i]['description']
        index1 = tab.iloc[i]['index']

        for j in range(i+1,len(tab)): # scorrere le righe interne alla razza
            breed2 = tab.iloc[j]['breed_primary']
            desc2 = tab.iloc[j]['description']
            index2 = tab.iloc[j]['index']

            if breed1 == breed2: #se uguale razza allora faccio il confronto
                if SequenceMatcher(None, desc1, desc2).ratio() >= 0.9: #se supera il 90% allora sono cloni e salvo i valori
                    duplicate[index1]= 'duplicate'
                    duplicate[index2]= 'duplicate of '+ str(index1)
            else: #razza diversa fermo j e mando avanti i
                break
            
            
    print(duplicate)

#questo funziona ma è molto lento

In [9]:
res1 = scannerDuplicate(duplicate_female2)
res1

{32156: 'duplicate', 32235: 'duplicate of 32156', 39499: 'duplicate', 39500: 'duplicate of 39499', 20300: 'duplicate', 20299: 'duplicate of 20300', 8256: 'duplicate', 8257: 'duplicate of 8256', 40039: 'duplicate', 40045: 'duplicate of 40039'}


In [10]:
# se nel ciclo dopo passo sopra ad una che è gia duplicata posso non ricontrollarla 
#(anchge se magari quelle dopo sono duplicate rispetto a questa in un modo leggermente diverso e con 
#la prima duplicazione non vengono viste perché < 90)

def scannerDuplicate_senzaDuplicate(tab): 
    duplicate2 = {}
    for i in range(len(tab)): # per scorrere tutte le righe
        breed1 = tab.iloc[i]['breed_primary']
        desc1 = tab.iloc[i]['description']
        index1 = tab.iloc[i]['index']
        
        if index1 not in duplicate2: #se è contenuto significa che è gia stato calcolato
            for j in range(i+1,len(tab)): # scorrere le righe interne alla razza
                breed2 = tab.iloc[j]['breed_primary']
                desc2 = tab.iloc[j]['description']
                index2 = tab.iloc[j]['index']

                if breed1 == breed2: #se uguale razza allora faccio il confronto
                    if SequenceMatcher(None, desc1, desc2).ratio() >= 0.9: #se supera il 90% allora sono cloni e salvo i valori
                        duplicate2[index1]= 'duplicate'
                        duplicate2[index2]= 'duplicate'
                else: #razza diversa fermo j e mando avanti i
                    break

        
    print(duplicate2)
# funziona e non ricalcola i duplicati, qualche ciclo in meno 

In [11]:
res2 = scannerDuplicate_senzaDuplicate(duplicate_female2)
res2

{32156: 'duplicate', 32235: 'duplicate', 39499: 'duplicate', 39500: 'duplicate', 20300: 'duplicate', 20299: 'duplicate', 8256: 'duplicate', 8257: 'duplicate', 40039: 'duplicate', 40045: 'duplicate'}


In [14]:
#proviamo a lanciare 21:05
dup_female = scannerDuplicate_senzaDuplicate(duplicate_female)
dup_female
# troppo lento 

KeyboardInterrupt: 